In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from keras import Sequential
from keras.src.layers import LSTM, Dense
from random import randint

In [ ]:
def get_df(data_size:int=500, path:str="..\Data\AAPL_stock_prices.csv", delimeter: str = ',', from_end: bool = True, date_column: str = 'Date', target_column: str = 'Close') -> tuple[np.ndarray, MinMaxScaler, int]:
    df = pd.read_csv(path, delimiter=delimeter)
    df = df.iloc[-data_size:, :] if from_end else df.iloc[:data_size, :]
    dates = pd.to_datetime(df[date_column])
    df.drop(columns=[date_column], inplace=True)
    df.index = dates

    target_column_index = df.columns.tolist().index(target_column)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df)
    return scaled_data, scaler, target_column_index

scaled_data, scaler, target_column_index = get_df()


In [ ]:
def create_dataset(data: np.ndarray, time_step: int=10):
    X, Y = [], []
    for i in range(len(data) - time_step):
        # Define the range of input sequences
        end_ix = i + time_step
        
        # Define the range of output sequences
        out_end_ix = end_ix + 1
        
        # Ensure that the dataset is within bounds
        if out_end_ix > len(data)-1:
            break
            
        # Extract input and output parts of the pattern
        seq_x, seq_y = data[i:end_ix], data[out_end_ix]
        
        # Append the parts
        X.append(seq_x)
        Y.append(seq_y)
    return np.array(X), np.array(Y), data.shape[1], time_step


X, Y, feature_number, time_step = create_dataset(data=scaled_data)

In [ ]:
# Split the data into training and testing sets
train_size = int(len(X) * 0.7)
test_size = len(X) - train_size
X_train, X_test = X[0:train_size], X[train_size:len(X)]
Y_train, Y_test = Y[0:train_size], Y[train_size:len(Y)]

In [ ]:
print(f"{X_train.shape=}, {Y_train.shape=}")
print(f"{X_test.shape=}, {Y_test.shape=}")
print(X_train[:1, 0, :])
print(X_test[:1, 0, :])
print(Y_train[:1, :])
print(Y_test[:1, :])

In [ ]:
def create_model(unit_one, unit_two, unit_three, unit_four, activation='linear', optimizer: str = 'adam', loss: str = 'mean_squared_error', batch_size=1, epochs=50) -> Sequential:
    # Create the LSTM model
    metrics: list[str]=['accuracy']
    input_shape: tuple = (10,6)
    model = Sequential()
    model.add(LSTM(unit_one, return_sequences=True, input_shape=input_shape, activation=activation))
    if unit_two > 0:
        model.add(LSTM(unit_two, return_sequences=True, input_shape=input_shape))
    if unit_three > 0:
        model.add(LSTM(unit_three, return_sequences=True, input_shape=input_shape))        
    model.add(LSTM(unit_four, return_sequences=False))
    model.add(Dense(1))
    model.compile(optimizer = optimizer, loss = loss, metrics=metrics)
    return model

In [ ]:
param_grid = {
    'model__optimizer': ['RMSprop', 'adam'],
    'model__loss': ['mean_squared_error'],
    'batch_size': [1, 2],
    'epochs': [50],
    'model__unit_one': [64, 128, 256],
    'model__unit_two': [64, 128],
    'model__unit_three': [64, 128],
    'model__unit_four': [64],
    'model__activation': ['relu', 'tanh', 'sigmoid','linear']
}

In [ ]:
lstm_regressor = KerasRegressor(model=create_model, verbose=1)
# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=lstm_regressor, param_grid=param_grid, scoring='neg_mean_absolute_error', error_score='raise', cv=5)
# Fit the grid search to the data
grid_search.fit(X_train, Y_train[:, target_column_index].reshape(-1,1))


with open("example.txt", "w") as file:
    # Write some content to the file
    file.write(f"{grid_search.best_params_=}, {grid_search.best_score_=}\n\n")
    file.write(f"{grid_search.best_estimator_=}\n\n")
    file.write(f"{grid_search.best_index_=}\n\n")
    file.write(f"{grid_search.scorer_=}\n\n")
    file.write(f"{grid_search.cv_results_=}\n\n")


print(grid_search.best_params_)

print(grid_search.best_score_)